In [1]:
import torch
import torch.nn as nn
import torch.onnx
import onnx
import onnxruntime
import numpy as np

In [2]:
from NN import Encoder_n_Decoder

In [3]:
F = torch.load("sound_ai.pt", weights_only=False)

print(F)

Encoder_n_Decoder(
  (encoder): Encoder(
    (norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (rnn): LSTM(32, 32, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (f): Sequential(
      (0): Linear(in_features=64, out_features=16, bias=True)
      (1): ReLU()
      (2): Linear(in_features=16, out_features=7, bias=True)
    )
  )
)


In [4]:
x = torch.randn(1,500,32)
dynamic_axes = {"x" : {0 : 'b', 1 : 'f'}, "y" : {0 : 'b'}}

torch.onnx.export(
    F,
    x,
    "sound_ai.onnx",
    export_params=True,
    opset_version=20,
    do_constant_folding=True,
    input_names=["x"],
    output_names=["y"],
    dynamic_axes=dynamic_axes
)

C:\ProgramData\anaconda3\Lib\site-packages\torch\onnx\symbolic_opset9.py:4277: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


In [5]:
onnx_F = onnx.load('sound_ai.onnx')

onnx.checker.check_model(onnx_F)

In [7]:
onnx_F = onnxruntime.InferenceSession('sound_ai.onnx', providers=['CPUExecutionProvider'])

np_x = np.random.randn(2,300,32).astype(np.float32)
tensor_x = torch.tensor(np_x, dtype=torch.float)

inputs = {onnx_F.get_inputs()[0].name : np_x}
outputs = onnx_F.run(None, inputs)

np_y = outputs[0]
tensor_y = F(tensor_x)

np.testing.assert_allclose(tensor_y.detach().numpy(), np_y, rtol=1e-03, atol=1e-05)